# Lecture 3: Social Choice Theory & Influences

**Topics**: Social Choice, Influences, Effects, Derivatives

**O'Donnell Chapters**: 2.1-2.2
**Notebook by: Gabriel Taboada**

---

## Key Concepts

1. **Social Choice Functions**: Boolean functions as voting systems
2. **Variable Influences**: $\text{Inf}_i[f] = \Pr[f(x) \neq f(x^{\oplus i})]$
3. **Derivative**: $D_i f(x) = f(x^{i \to 1}) - f(x^{i \to 0})$
4. **Banzhaf Power Index**: Voting power from Boolean analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import boofun as bf
from boofun.visualization import BooleanFunctionVisualizer, plot_function_comparison

np.random.seed(42)
plt.rcParams['figure.figsize'] = (12, 5)
print("✓ Libraries loaded (including visualization module)")

## 1. Social Choice Functions

A **social choice function** (SCF) is a Boolean function $f: \{-1, +1\}^n \to \{-1, +1\}$ where:
- Each input $x_i \in \{-1, +1\}$ represents voter $i$'s preference
- Output $f(x)$ is the "society's decision"

### Properties of Good Voting Rules
- **Unanimous**: $f(1,1,...,1) = 1$ and $f(-1,-1,...,-1) = -1$
- **Odd**: $f(-x) = -f(x)$ (no bias toward either outcome)
- **Monotone**: If more voters prefer $+1$, outcome shouldn't switch to $-1$
- **Symmetric**: All voters treated equally

In [ ]:
# Common social choice functions
n = 5

# 1. Majority Vote (the most "fair" symmetric rule)
maj = bf.majority(n)

# 2. Dictator (one voter decides everything)
dictator = bf.dictator(n, 0)  # Dictator on x₀

# 3. AND (unanimous required)
and_f = bf.AND(n)

# 4. OR (at least one supporter needed)
or_f = bf.OR(n)

print("Social Choice Functions (n=5 voters):")
print(f"{'Function':15} | Balanced | Monotone | Symmetric")
print("-" * 55)
for name, f in [("Majority", maj), ("Dictator", dictator), ("AND", and_f), ("OR", or_f)]:
    print(f"{name:15} | {str(f.is_balanced()):8} | {str(f.is_monotone()):8} | {str(f.is_symmetric()):8}")

## 2. Variable Influences

The **influence** of variable $i$ measures how often flipping voter $i$'s vote changes the outcome:

$$\text{Inf}_i[f] = \Pr_{x \sim \{-1,+1\}^n}[f(x) \neq f(x^{\oplus i})]$$

### Fourier Formula
$$\text{Inf}_i[f] = \sum_{S: i \in S} \hat{f}(S)^2$$

In [ ]:
# Compare influences across different voting rules
functions = {
    "Majority": bf.majority(n),
    "Dictator x₀": bf.dictator(n, 0),
    "Parity": bf.parity(n),
    "AND": bf.AND(n),
}

fig, axes = plt.subplots(1, len(functions), figsize=(14, 4))

for ax, (name, f) in zip(axes, functions.items()):
    # Direct API: f.influences()
    influences = f.influences()
    
    ax.bar(range(n), influences, color='steelblue', alpha=0.7)
    ax.set_title(f"{name}\nTotal I[f]: {sum(influences):.3f}")
    ax.set_xlabel("Voter")
    ax.set_ylabel("Influence")
    ax.set_ylim(0, 1.1)
    ax.set_xticks(range(n))

plt.tight_layout()
plt.show()

print("\n📊 Key Observations:")
print("   - Majority: All voters equally influential (symmetric)")
print("   - Dictator: One voter has ALL the influence")
print("   - Parity: All voters equally influential, but NOT monotone")
print("   - AND: All voters equal, but very low individual influence")

## 3. Total Influence and Spectral Formula

The **total influence** is:
$$I[f] = \sum_{i=1}^n \text{Inf}_i[f] = \sum_{S} |S| \cdot \hat{f}(S)^2$$

This measures the overall "sensitivity" of the function.

In [ ]:
# Verify the spectral formula for total influence
def total_influence_spectral(f):
    """Compute I[f] = Σ |S| · f̂(S)² via Fourier."""
    # Direct API: f.fourier()
    fourier = f.fourier()
    n = f.n_vars
    
    total = 0.0
    for s in range(2**n):
        subset_size = bin(s).count('1')
        total += subset_size * fourier[s]**2
    return total

print("Verifying Total Influence Formula: I[f] = Σ|S|·f̂(S)²")
print("="*55)

for name, f in functions.items():
    # Direct API: f.total_influence()
    inf_direct = f.total_influence()
    inf_spectral = total_influence_spectral(f)
    
    match = "✓" if abs(inf_direct - inf_spectral) < 1e-10 else "✗"
    print(f"{name:15}: Direct={inf_direct:.4f}, Spectral={inf_spectral:.4f} {match}")

## 4. Visualizing Voting Power

The `boofun` visualization module can compare influence patterns across different voting rules.

In [ ]:
# Compare influence patterns using visualization
# This shows how voting power is distributed in different systems

voting_rules = {
    "Majority": bf.majority(n),
    "Dictator": bf.dictator(n, 0),
    "Weighted": bf.weighted_majority([3, 2, 2, 1, 1]),  # Weighted voting
}

# Use the comparison plot
fig = plot_function_comparison(voting_rules, metric="influences")

print("\n📊 Influence Comparison:")
print("   - Majority: Equal power for all voters (democratic)")
print("   - Dictator: One voter has all the power")
print("   - Weighted: Power proportional to weight")

# Also show noise stability comparison
fig = plot_function_comparison(voting_rules, metric="fourier")

print("\n   Fourier weight comparison shows spectral structure")

## Summary

### Key Takeaways

1. **Social Choice as Boolean Functions**: Voting rules are monotone, odd Boolean functions

2. **Influence = Voting Power**: $\text{Inf}_i[f]$ measures how often voter $i$ is "pivotal"

3. **Spectral Formula**: $\text{Inf}_i[f] = \sum_{S \ni i} \hat{f}(S)^2$

4. **Total Influence**: $I[f] = \sum_S |S| \cdot \hat{f}(S)^2$

### boofun Usage (Direct API)
```python
# Compute influences - direct methods!
influences = f.influences()
total_inf = f.total_influence()
fourier = f.fourier()

# Create voting functions
maj = bf.majority(n)
dict_f = bf.dictator(n, i)
```